In [2]:
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE, KMeansSMOTE, SVMSMOTE
from imblearn.under_sampling import CondensedNearestNeighbour, EditedNearestNeighbours,  \
                                    RepeatedEditedNearestNeighbours, AllKNN, NearMiss, NeighbourhoodCleaningRule, OneSidedSelection, TomekLinks
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score 
from imblearn.combine import SMOTETomek, SMOTEENN

import random
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv('../datasets/BankChurners.csv')
list = ['Attrition_Flag', 'Total_Trans_Ct', 'Total_Trans_Amt', 'Total_Revolving_Bal', 'Total_Ct_Chng_Q4_Q1', 'Contacts_Count_12_mon', 'Total_Relationship_Count', 'Months_Inactive_12_mon', 'Months_on_book']
data = data[list]
data

,Attrition_Flag,Total_Trans_Ct,Total_Trans_Amt,Total_Revolving_Bal,Total_Ct_Chng_Q4_Q1,Contacts_Count_12_mon,Total_Relationship_Count,Months_Inactive_12_mon,Months_on_book
0,Existing Customer,42,1144,777,1.625,3,5,1,39
1,Existing Customer,33,1291,864,3.714,2,6,1,44
2,Existing Customer,20,1887,0,2.333,0,4,1,36
3,Existing Customer,20,1171,2517,2.333,1,3,4,34
4,Existing Customer,28,816,0,2.500,0,5,1,21
...,...,...,...,...,...,...,...,...,...
10122,Existing Customer,117,15476,1851,0.857,3,3,2,40
10123,Attrited Customer,69,8764,2186,0.683,3,4,2,25
10124,Attrited Customer,60,10291,0,0.818,4,5,3,36
10125,Attrited Customer,62,8395,0,0.722,3,4,3,36


In [5]:
object_columns = data.select_dtypes('object').columns

for i in object_columns:

    lb = LabelEncoder()
    lb.fit(data[i])
    data[i] = lb.transform(data[i])
    
    print(f'category : {np.unique(data[i])}\nclasses : {lb.classes_}\n')

input = data.iloc[:,1:]
target = data.iloc[:,0]

category : [0 1]
classes : ['Attrited Customer' 'Existing Customer']



In [6]:
## Sampling 전부

columns = ['f1_score', 'recall', 'precision', 'accuracy']
df = pd.DataFrame(columns=columns)

sampling = [EditedNearestNeighbours,RepeatedEditedNearestNeighbours, AllKNN, NearMiss,NeighbourhoodCleaningRule, OneSidedSelection,TomekLinks, SMOTE, BorderlineSMOTE, ADASYN, SMOTETomek, SMOTEENN]

random.seed(42)

for i in sampling:
    x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

    ss = StandardScaler()
    x_train_ss = ss.fit_transform(x_train)
    x_test_ss = ss.fit_transform(x_test)

    sampling = i()
    x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

    rf = RandomForestClassifier(random_state=42)
    rf.fit(x_train_ss, y_train)
    pred = rf.predict(x_test_ss)

    acc = accuracy_score(y_test, pred)
    recall = recall_score(y_test, pred)
    precision = precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)

    df.loc[sampling] = [f1, recall, precision, acc]

print("RandomForest Model's score by sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False, inplace=True)
df.head(10)

RandomForest Model's score by sampling


,f1_score,recall,precision,accuracy
NeighbourhoodCleaningRule(),0.974374,0.973514,0.975236,0.957058
TomekLinks(),0.974329,0.982931,0.965876,0.956565
OneSidedSelection(),0.974075,0.984108,0.964245,0.956071
EditedNearestNeighbours(),0.973980,0.969394,0.978610,0.956565
SMOTE(),0.970614,0.962331,0.979042,0.951135
AllKNN(),0.970362,0.963508,0.977313,0.950642
SMOTETomek(),0.970273,0.960565,0.980180,0.950642
RepeatedEditedNearestNeighbours(),0.966974,0.956445,0.977738,0.945212
BorderlineSMOTE(),0.963696,0.945262,0.982864,0.940276
ADASYN(),0.962673,0.941142,0.985213,0.938796


In [7]:
## Under Sampling 만

columns = ['f1_score', 'recall', 'precision', 'accuracy']
df = pd.DataFrame(columns=columns)

sampling = [EditedNearestNeighbours, RepeatedEditedNearestNeighbours, AllKNN, NearMiss,NeighbourhoodCleaningRule, OneSidedSelection,TomekLinks]

random.seed(42)

for i in sampling:
    
    x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

    ss = StandardScaler()
    x_train_ss = ss.fit_transform(x_train)
    x_test_ss = ss.fit_transform(x_test)

    sampling = i()
    x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

    rf = RandomForestClassifier(random_state=42)
    rf.fit(x_train_ss, y_train)
    pred = rf.predict(x_test_ss)

    acc = accuracy_score(y_test, pred)
    recall = recall_score(y_test, pred)
    precision = precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    
    df.loc[sampling] = [f1, recall, precision, acc]

print("RandomForest Model's score by Under sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False)

RandomForest Model's score by Under sampling


,f1_score,recall,precision,accuracy
OneSidedSelection(),0.975510,0.984697,0.966493,0.958539
NeighbourhoodCleaningRule(),0.974374,0.973514,0.975236,0.957058
TomekLinks(),0.974329,0.982931,0.965876,0.956565
EditedNearestNeighbours(),0.973980,0.969394,0.978610,0.956565
AllKNN(),0.970362,0.963508,0.977313,0.950642
RepeatedEditedNearestNeighbours(),0.966974,0.956445,0.977738,0.945212
NearMiss(),0.895541,0.827546,0.975711,0.838105


In [8]:
## Over Sampling 만

columns = ['f1_score', 'recall', 'precision', 'accuracy']
df = pd.DataFrame(columns=columns)

sampling = [SMOTE, BorderlineSMOTE, ADASYN]
strategy_ = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

random.seed(42)

for i in sampling:
    
    for strategy in strategy_:
        x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

        ss = StandardScaler()
        x_train_ss = ss.fit_transform(x_train)
        x_test_ss = ss.fit_transform(x_test)

        sampling = i(sampling_strategy=strategy)
        x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

        rf = RandomForestClassifier(random_state=42)
        rf.fit(x_train_ss, y_train)
        pred = rf.predict(x_test_ss)

        acc = accuracy_score(y_test, pred)
        recall = recall_score(y_test, pred)
        precision = precision_score(y_test, pred)
        f1 = f1_score(y_test, pred)
        
        df.loc[i.__name__+'('+str(strategy)+')'] = [f1, recall, precision, acc]

print("RandomForest Model's score by Over sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False, inplace=True)
df.head(10)

RandomForest Model's score by Over sampling


,f1_score,recall,precision,accuracy
SMOTE(0.4),0.976197,0.977634,0.974765,0.960020
ADASYN(0.3),0.975294,0.975868,0.974721,0.958539
BorderlineSMOTE(0.5),0.975221,0.972925,0.977528,0.958539
BorderlineSMOTE(0.3),0.975095,0.979400,0.970828,0.958045
SMOTE(0.6),0.974934,0.972925,0.976950,0.958045
ADASYN(0.4),0.974934,0.972925,0.976950,0.958045
SMOTE(0.5),0.974329,0.971748,0.976923,0.957058
BorderlineSMOTE(0.4),0.974298,0.970571,0.978055,0.957058
SMOTE(0.3),0.974254,0.979988,0.968586,0.956565
SMOTE(0.7),0.973053,0.967039,0.979142,0.955084


In [9]:
## Combine Sampling 만

columns = ['f1_score', 'recall', 'precision', 'accuracy']
df = pd.DataFrame(columns=columns)

sampling = [SMOTETomek, SMOTEENN]

random.seed(42)

for i in sampling:

    x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=42)

    ss = StandardScaler()
    x_train_ss = ss.fit_transform(x_train)
    x_test_ss = ss.fit_transform(x_test)

    sampling = i()
    x_train_ss, y_train = sampling.fit_resample(x_train_ss, y_train)

    rf = RandomForestClassifier(random_state=42)
    rf.fit(x_train_ss, y_train)
    pred = rf.predict(x_test_ss)

    acc = accuracy_score(y_test, pred)
    recall = recall_score(y_test, pred)
    precision = precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    
    df.loc[sampling] = [f1, recall, precision, acc]

print("RandomForest Model's score by Combine sampling")
df.sort_values(['f1_score', 'recall', 'precision', 'accuracy'], ascending=False)

RandomForest Model's score by Combine sampling


,f1_score,recall,precision,accuracy
SMOTETomek(),0.969625,0.958211,0.981314,0.949654
SMOTEENN(),0.955704,0.927016,0.986224,0.927937
